In [356]:
import warnings

import numpy as np
from numba import njit
from scipy.optimize import minimize

In [2]:
@njit
def tsconv(a, b):
    na = len(a)
    nb = len(b)
    
    nab = na + nb - 1
    ab = np.zeros(nab)
    
    for i in range(na):
        for j in range(nb):
            ab[i + j] += a[i] * b[j]
            
    return ab

In [3]:
x = np.arange(1, 11)

In [4]:
%timeit tsconv(x, x)

675 ns ± 0.991 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [38]:
expected_tsconv = np.array([
    1, 4, 10, 20, 35, 56, 84, 120, 165, 220, 264,
    296, 315, 320, 310, 284, 241, 180, 100
])

In [6]:
np.testing.assert_array_almost_equal(expected_tsconv, tsconv(x, x))

In [40]:
@njit
def inclu2(np_, xnext, xrow, ynext, d, rbar, thetab):
    for i in range(np_):
        xrow[i] = xnext[i]
    
    ithisr = 0
    for i in range(np_):
        if xrow[i] != 0.:
            xi = xrow[i]
            di = d[i]
            dpi = di + xi * xi
            d[i] = dpi
            cbar = di / dpi
            sbar = xi / dpi
            for k in range(i + 1, np_):
                xk = xrow[k]
                rbthis = rbar[ithisr]
                xrow[k] = xk - xi * rbthis
                rbar[ithisr] = cbar * rbthis + sbar * xk
                ithisr += 1
            xk = ynext
            ynext = xk - xi * thetab[i]
            thetab[i] = cbar * thetab[i] + sbar * xk
            if di == 0.:
                return
        else:
            ithisr = ithisr + np_ - i - 1

In [41]:
@njit
def getQ0(phi, theta):
    p = len(phi)
    q = len(theta)
    r = max(p, q + 1)
    
    np_ = r * (r + 1) // 2
    nrbar = np_ * (np_ - 1) // 2
    
    V = np.zeros(np_)
    ind = 0
    for j in range(r):
        vj = 0.
        if j == 0:
            vj = 1.
        elif j - 1 < q:
            vj = theta[j - 1]
        
        for i in range(j, r):
            vi = 0.
            if i == 0:
                vi = 1.0
            elif i - 1 < q:
                vi = theta[i - 1]
            V[ind] = vi * vj
            ind += 1
            
    res = np.zeros((r, r))
    res = res.flatten()
    
    if r == 1:
        if p == 0:
            res[0] = 1.
        else:
            res[0] = 1. / (1. - phi[0] * phi[0])
        
        res = res.reshape((r, r))
        return res
    
    if p > 0:
        rbar = np.zeros(nrbar)
        thetab = np.zeros(np_)
        xnext = np.zeros(np_)
        xrow = np.zeros(np_)
        
        ind = 0
        ind1 = -1
        npr = np_ - r
        npr1 = npr + 1
        indj = npr
        ind2 = npr - 1
        
        for j in range(r):
            phij = phi[j] if j < p else 0.
            xnext[indj] = 0.
            indj += 1
            indi = npr1 + j
            for i in range(j, r):
                ynext = V[ind]
                ind += 1
                phii = phi[i] if i < p else 0.
                if j != r - 1:
                    xnext[indj] = -phii
                    if i != r - 1:
                        xnext[indi] -= phij
                        ind1 += 1
                        xnext[ind1] = -1.
                xnext[npr] = -phii * phij
                ind2 += 1
                if ind2 >= np_:
                    ind2 = 0
                xnext[ind2] += 1.
                inclu2(np_, xnext, xrow, ynext, res, rbar, thetab)
                xnext[ind2] = 0.
                if i != r - 1:
                    xnext[indi] = 0.
                    indi += 1
                    xnext[ind1] = 0.
            
        ithisr = nrbar - 1
        im = np_ - 1
        for i in range(np_):
            bi = thetab[im]
            jm = np_ - 1
            for j in range(i):
                bi -= rbar[ithisr] * res[jm]
                ithisr -= 1
                jm -= 1
            res[im] = bi
            im -= 1
        
        # Now reorder p
        ind = npr
        for i in range(r):
            xnext[i] = res[ind]
            ind += 1
        ind = np_ - 1
        ind1 = npr - 1
        for i in range(npr):
            res[ind] = res[ind1]
            ind -= 1
            ind1 -= 1
        for i in range(r):
            res[i] = xnext[i]
    else:
        indn = np_
        ind = np_
        for i in range(r):
            for j in range(i + 1):
                ind -= 1
                res[ind] = V[ind]
                if j != 0:
                    indn -= 1
                    res[ind] += res[ind]
        
    # Unpack to a full matrix
    ind = np_
    for i in range(r - 1, 0, -1):
        for j in range(r - 1, i - 1, -1):
            ind -= 1
            res[r * i + j] = res[ind]

    for i in range(r - 1):
        for j in range(i + 1, r):
            res[i + r * j] = res[j + r * i]
    
    res = res.reshape((r, r))
    return res

In [42]:
expected_getQ0 = np.array([
       [ -3.07619732,   1.11465544,   2.11357369,   3.15204201,
          4.19013718,   5.22823588,   6.26633453,   7.30443355,
          8.34249459,   9.38458115,  10.        ],
       [  1.11465544,  -3.22931088,   1.92416552,   2.84615733,
          3.80807237,   4.76961073,   5.73115265,   6.69269418,
          7.65427405,   8.61179041,  10.        ],
       [  2.11357369,   1.92416552,  -0.37881633,   5.73654439,
          7.62116681,   9.54570541,  11.46986742,  13.39403227,
         15.31827268,  17.23450038,  20.        ],
       [  3.15204201,   2.84615733,   5.73654439,   4.39470753,
         11.47233269,  14.31920899,  17.20600158,  20.0924165 ,
         22.9789482 ,  25.85347889,  30.        ],
       [  4.19013718,   3.80807237,   7.62116681,  11.47233269,
         11.09276725,  19.13264974,  22.94178352,  26.79083216,
         30.63965504,  34.47249261,  40.        ],
       [  5.22823588,   4.76961073,   9.54570541,  14.31920899,
         19.13264974,  19.71534157,  28.71748151,  33.48887095,
         38.30036514,  43.09150596,  50.        ],
       [  6.26633453,   5.73115265,  11.46986742,  17.20600158,
         22.94178352,  28.71748151,  30.2624308 ,  40.22682604,
         45.96069867,  51.71052289,  60.        ],
       [  7.30443355,   6.69269418,  13.39403227,  20.0924165 ,
         26.79083216,  33.48887095,  40.22682604,  42.73402992,
         53.66094562,  60.32916003,  70.        ],
       [  8.34249459,   7.65427405,  15.31827268,  22.9789482 ,
         30.63965504,  38.30036514,  45.96069867,  53.66094562,
         57.13074521,  68.98805242,  80.        ],
       [  9.38458115,   8.61179041,  17.23450038,  25.85347889,
         34.47249261,  43.09150596,  51.71052289,  60.32916003,
         68.98805242,  73.38026771,  90.        ],
       [ 10.        ,  10.        ,  20.        ,  30.        ,
         40.        ,  50.        ,  60.        ,  70.        ,
         80.        ,  90.        , 100.        ]]
)

In [43]:
np.testing.assert_array_almost_equal(expected_getQ0, getQ0(x, x))

In [44]:
%timeit getQ0(x, x)

12.9 µs ± 136 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


- armaCSS
    - ARIMA_transPars
        - fixed, params narma + ncxreg
        - mask = is.na(fixed)
        - coef = as.double(fixed)
        - arma <- as.integer(c(order[-2L], seasonal$order[-2L], seasonal$period,
                         order[2L], seasonal$order[2L]))
    narma <- sum(arma[1L:4L])
    - ARIMA_CSS


In [137]:
@njit
def partrans(p, raw, new):
    new[:] = np.tanh(raw)
    work = new.copy()
    
    for j in range(1, p):
        a = new[j]
        for k in range(j):
            work[k] -= a * new[j - k - 1]
        new[:j] = work[:j]

In [259]:
@njit
def arima_transpar(params_in, arma, trans):
    #TODO check trans=True results
    mp, mq, msp, msq, ns = arma[0], arma[1], arma[2], arma[3], arma[4]
    p = mp + ns * msp
    q = mq + ns * msq
    
    phi = np.zeros(p)
    theta = np.zeros(q)
    
    params = params_in.copy()
    
    if trans:
        if mp > 0:
            print(params)
            partrans(mp, params_in, params)
            print(params)
    
        v = mp + mq
        if msp > 0:
            print(params)
            #params += v
            partrans(msp, params_in + v, params + v)
            #params -= v
            print(params)
    
    if ns > 0:
        phi[:mp] = params[:mp]
        phi[mp:] = 0.
        theta[:mq] = params[mp:mp + mq]
        theta[mq:] = 0.
        
        for j in range(msp):
            phi[(j + 1) * ns - 1] += params[j + mp + mq]
            for i in range(mp):
                phi[(j + 1) * ns + i] -= params[i] * params[j + mp + mq]
        
        for j in range(msq):
            theta[(j + 1) * ns - 1] += params[j + mp + mq + msp]
            for i in range(mq):
                theta[(j + 1) * ns + i] += params[i + mp] * params[j + mp + mq + msp]
    else:
        phi[:mp] = params[:mp]
        theta[:mq] = theta[mp:mp + mq]
        
    return phi, theta

In [260]:
expected_arima_transpar_f = (
    np.array([ 0.5 ,  1.  , -0.25,  0.25, -0.25, -0.25]),
    np.array([0.5 , 1.  , 0.25, 0.75, 0.25, 0.25])
)

In [261]:
params = np.repeat(.5, 10)
arma = np.ones(5, dtype=np.integer) * 2
for exp, calc in zip(expected_arima_transpar_f, arima_transpar(params, arma, False)):
    np.testing.assert_array_almost_equal(exp, calc)

In [262]:
%timeit arima_transpar(params, arma, False)

1.06 µs ± 7.81 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [263]:
@njit
def arima_css(y, arma, phi, theta, ncond):
    n = len(y)
    p = len(phi)
    q = len(theta)
    nu = 0
    ssq = 0.0
    
    w = y.copy()
    
    for i in range(arma[5]):
        for l in range(n - 1, 0, -1):
            w[l] -= w[l - 1]
    
    ns = arma[4]
    for i in range(arma[6]):
        for l in range(n - 1, ns - 1, -1):
            w[l] -= w[l - ns]
    
    resid = np.empty(n)
    resid[:ncond] = 0.
    for l in range(ncond, n):
        tmp = w[l]
        for j in range(p):
            if l - j - 1 < 0:
                continue
            tmp -= phi[j] * w[l - j - 1]
            
        for j in range(min(l - ncond, q)):
            if l - j - 1 < 0:
                continue
            tmp -= theta[j] * resid[l - j - 1]
            
        resid[l] = tmp
        
        if not np.isnan(tmp):
            nu += 1
            ssq += tmp * tmp
    
    res = ssq / nu
    
    return res, resid

In [236]:
%%timeit 
arima_css(np.arange(1, 11), 
          np.array([0,0,0,0,0,0,0], dtype=np.int32),
          expected_arima_transpar_f[0],
          expected_arima_transpar_f[1], 
          3)

The slowest run took 8.48 times longer than the fastest. This could mean that an intermediate result is being cached.
5.82 µs ± 7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [237]:
%%timeit 
arima_css(np.arange(1, 11), 
          np.array([0,0,0,0,0,0,0], dtype=np.int32),
          expected_arima_transpar_f[0],
          expected_arima_transpar_f[1], 
          3)

2.49 µs ± 16 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [625]:
#@njit
def make_arima(phi, theta, delta, kappa = 1e6, tol = np.finfo(float).eps):
    # check nas phi
    # check nas theta
    p = len(phi)
    q = len(theta)
    r = max(p, q + 1)
    d = len(delta)
    
    rd = r + d
    Z = np.concatenate((np.array([1.]), np.zeros(r - 1), delta))
    T = np.zeros((rd, rd))
    
    if p > 0:
        T[:p, 0] = phi
    if r > 1:
        ind = np.arange(1, r)
        #T[range(r), range(r)] = 1
        T[ind - 1, ind] = 1
        #np.diagonal(T, 1) = 1]
        #T[np.diag_indices_from(T, 1)] = 1

    if d > 0:
        T[r] = Z
        if d > 1:
            ind = r + np.arange(1, d)
            T[ind, ind - 1] = 1

    if q < r - 1:
        theta = np.concatenate((theta, np.zeros(r - 1 - q)))

    R = np.concatenate((np.array([1.]), theta, np.arange(d)))
    V = R * R.reshape(-1, 1)
    h = 0.
    a = np.zeros(rd)
    Pn = np.zeros((rd, rd))
    P = np.zeros((rd, rd))
    
    if r > 1:
        Pn[:r, :r] = getQ0(phi, theta)
    else:
        Pn[0, 0] = 1 / (1 - phi[0] ** 2) if p > 0 else 1.
    
    if d > 0:
        ind = r + np.arange(d)
        Pn[ind, ind] = kappa
        
    res = {
        'phi': phi, 
        'theta': theta, 
        'delta': delta, 
        'Z': Z, 
        'a': a, 
        'P': P, 
        'T': T, 
        'V': V, 
        'h': h, 
        'Pn': Pn
    }
    
    return res

In [626]:
make_arima(np.arange(1, 5), np.arange(1, 5), np.arange(1, 5))['T']

array([[1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [2., 0., 1., 0., 0., 0., 0., 0., 0.],
       [3., 0., 0., 1., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 2., 3., 4.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [ ]:
def arima_like(y, phi, theta, delta, a, P, Pn, up):
    n = len(y)
    rd = len(a)
    p = len(phi)
    q = len(theta)
    d = len(delta)
    r = rd - d
    
    sumlog = 0.
    ssq = 0.
    nu = 0
    
    anew = np.empty(rd)
    M = np.empty(rd)
    if d > 0:
        mm = np.empty(rd * rd)
        
    for l in range(n):
        for i in range(r):
            tmp = a[i + 1] if i < r - 1 else 0.
            if i < p:
                tmp += phi[i] * a[0]
            anew[i] = tmp
        if d > 0:
            for i in range(r + 1, rd):
                anew[i] = a[i - 1]
            tmp = a[0]
            for i in range(d):
                tmp += delta[i] * a[r + 1]
            anew[r] = tmp
        
        if l > up:
            if d == 0:
                for i in range(r):
                    vi = 0.
                    if i == 0:
                        vi = 1.
                    elif i - 1 < q:
                        vi = theta[i - 1]
                    for j in range(r):
                        tmp = 0.
                        if j == 0:
                            tmp = vi
                        elif j - 1 < q:
                            tmp = vi * theta[j - 1]
                        if i < p and j < p:
                            tmp += phi[i] * phi[j] * P[0]
                        if i < r - 1 and j < r -1:
                            tmp += P[i + 1 + r * (j + 1)]
                        if i < p and j < r - 1:
                            tmp += phi[i] * P[j + 1]
                        if j < p and i < r -1:
                            tmp += phi[j] * P[i + 1]
                        Pnew[i + r * j] = tmp
            else:
                # mm = TP
                for i in range(r):
                    for j in range(rd):
                        tmp = 0.
                        if i < p:
                            tmp += phi[i] * P[rd * j]
                        if i < r - 1:
                            tmp += P[i + 1 + rd * j]
                        mm[i + rd * j] = tmp
                for j in range(rd):
                    tmp = P[rd * j]
                    for k in range(d):
                        tmp += delta[k] * P[r + k + rd * j]
                    mm[r + rd * j] = tmp
                for i in range(1, d):
                    for j in range(rd):
                        mm[r + i + rd * j] = P[r + i - 1 + rd * j]
                
                # Pnew = mmT'
                for i in range(r):
                    for j in range(rd):
                        tmp = 0.
                        if i < p:
                            tmp += phi[i] * mm[j]
                        if i < r - 1:
                            tmp += mm[rd * (i + 1) + j]
                        Pnew[j + rd * i] = tmp
                for j in range(rd):
                    tmp = mm[j]
                    for k in range(d):
                        tmp += delta[k] * mm[rd * (r + k) + j]
                    Pnew[rd * r + j] = tmp
                for i in range(1, d):
                    for j in range(rd):
                        Pnew[rd * (r + i) + j] = mm[rd * (r + 1 - i) + j]
                
                for i in range(q + 1):
                    vi = 1. if i == 0 else theta[i - 1]
                    for j in range(q + 1):
                        Pnew[i + rd * j] += vi * (1. if j == 0 else theta[j - 1])
    
        if ~np.isnan(y[l]):
            
                        
                
                         

In [564]:
def arima(x: np.ndarray,
          order = (0, 0, 0),
          seasonal = {'order': (0, 0, 0), 'period': 1},
          xreg = None,
          include_mean = False,
          transform_pars = False,
          fixed = None,
          method = 'CSS',
          optim_method = 'BFGS',
          kappa = 1e6):
    # check x:
    # - size, should be 1
    # - is numeric
    n = len(x)
    
    # check order and seasonal
    # - order of size 3 and int
    # - seasonal
    #   - dict with order of size 3 and int
    #   - if period is null, na or 0 then period = freq(x)
    #   - default frequency of non defined ts is 1 in R
    
    #fixed
    #mask 
    arma = (*order[::2], 
            *seasonal['order'][::2],
            seasonal['period'],
            order[1],
            seasonal['order'][1])
    narma = sum(arma[:4])
    
    # xtsp = init x, end x and frequency
    # tsp(x) = None
    Delta = np.array([1.]) 
    for i in range(order[1]):
        Delta = tsconv(Delta, [1., -1.]) 
    
    for i in range(seasonal['order'][1]):
        Delta = tsconv(Delta, [1] + list(range(0, seasonal['period'] - 1)) + [-1])
    Delta = - Delta[1:]
    nd = order[1] + seasonal['order'][1]
    n_used = (~np.isnan(x)).sum() - len(Delta)
    
    if xreg is None:
        ncxreg = 0
    else:
        if xreg.shape[0] != n:
            raise Exception('lengths of `x` and `xreg` do not match')
        
        ncxreg = xreg.shape[1]
        
    if include_mean and (nd == 0):
        intercept = np.arange(1, n + 1).reshape(-1, 1)
        if xreg is None:
            xreg = intercept
        else:
            xreg = np.concatenate([intercept, xreg])
            ncxreg += 1
            
    # check nas for method CSS-ML
    
    
    if method in ['CSS']:
        ncond = order[1] + seasonal['order'][1] * seasonal['period']
        ncond1 = order[0] + seasonal['order'][0] * seasonal['period']
        ncond = ncond + ncond1
    else:
        ncond = 0
        
    if fixed is None:
        fixed = [np.nan for _ in range(narma + ncxreg)]
    else:
        if len(fixed) != narma + ncxreg:
            raise Exception('wrong length for `fixed`')
    mask = np.isnan(fixed)
    
    no_optim = not any(mask)
    
    if no_optim:
        transform_pars = False
        
    if transform_pars:
        ind = arma[0] + arma[1] + list(range(arma[2]))
        # check masks and more
        
    init0 = np.zeros(narma)
    parscale = np.ones(narma)
    
    # xreg processing
    # nused exception
    # not null init
    
    
    def arma_css_op(p):
        phi, theta = arima_transpar(p, arma, False)
        res, resid = arima_css(x, arma, phi, theta, ncond)
        
        return 0.5 * np.log(res)
    
    coef = np.array(fixed)
    # parscale definition, think about it, scipy doesnt use it
    
    if method == 'CSS':
        # add if no_optim result
        res = minimize(arma_css_op, init0, method=optim_method)
        
        if res.status > 0:
            warnings.warn(
                f'possible convergence problem: minimize gave code {res.status}]'
            )
            
        coef[mask] = res.x
        phi, theta = arima_transpar(coef, arma, False)
        mod = make_arima(phi, theta, Delta, kappa)
        # arimaSS
        sigma2, resid = arima_css(x, arma, phi, theta, ncond)
        var = np.empty() if no_optim else res.hess_inv / n_used
        
    value = 2 * n_used * res.fun + n_used + n_used * np.log(2 * np.pi)
    aic = value + 2 * sum(mask) + 2 if method != 'CSS' else np.nan
    
    ans = {
        'coef': coef, 
        'sigma2': sigma2, 
        'var_coef': var, 
        'mask': mask,
        'loglik': -0.5 * value, 
        'aic': aic, 
        'arma': arma,
        'residuals': resid, 
        #'series': series,
        'code': res.status, 
        'n_cond': ncond, 
        'nobs': n_used,
        'model': mod
    }
    
    
    return ans

In [599]:
res = arima(np.arange(1, 101), order=(1, 0, 1))

In [600]:
np.sqrt(res['var_coef'])

/var/folders/7x/1l3vkh3x4_q3s4r36b60s8f00000gn/T/ipykernel_17989/2960352552.py:1: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(res['var_coef'])


array([[0.00091795,        nan],
       [       nan, 0.03535651]])

In [603]:
@njit
def kalman_forecast(n, Z, a, P, T, V, h):
    p = len(a)
    
    anew = np.empty(p)
    Pnew = np.empty(p * p)
    mm = np.empty(p * p)
    forecasts = np.empty(n)
    se = np.empty(n)
    
    a = a.flatten()
    P = P.flatten()
    T = T.flatten()
    V = V.flatten()
    
    for l in range(n):
        fc = 0.
        for i in range(p):
            tmp = 0.
            for k in range(p):
                tmp += T[i + p * k] * a[k]
            anew[i] = tmp
            fc += tmp * Z[i]
            
        for i in range(p):
            a[i] = anew[i]
        forecasts[l] = fc
    
        for i in range(p):
            for j in range(p):
                tmp = 0.
                for k in range(p):
                    tmp += T[i + p * k] * P[k + p * j]
                mm[i + p * j] = tmp

        for i in range(p):
            for j in range(p):
                tmp = V[i + p * j]
                for k in range(p):
                    tmp += mm[i + p * k] * T[j + p * k]
                Pnew[i + p * j] = tmp

        tmp = h
        for i in range(p):
            for j in range(p):
                P[i + j * p] = Pnew[i + j * p]
                tmp += Z[i] * Z[j] * P[i + j * p]
                
        se[l] = tmp

    return forecasts, se

In [604]:
kalman_forecast(10, 
                *(res['model'][var] for var in ['Z', 'a', 'P', 'T', 'V', 'h']))

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([ 1.        ,  2.03024641,  3.09165406,  4.18516549,  5.3117517 ,
         6.4724131 ,  7.66818034,  8.90011524, 10.16931174, 11.47689687]))

In [591]:
%%timeit 
kalman_forecast(10, 
                *(res['model'][var] for var in ['Z', 'a', 'P', 'T', 'V', 'h']))

144 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [594]:
%%timeit 
kalman_forecast(10, 
                *(res['model'][var] for var in ['Z', 'a', 'P', 'T', 'V', 'h']))

2.8 µs ± 17.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [577]:
res['model']

{'phi': array([1.01501054]),
 'theta': array([0.87138826]),
 'delta': array([], dtype=float64),
 'Z': array([1., 0.]),
 'a': array([0., 0.]),
 'P': array([[0., 0.],
        [0., 0.]]),
 'T': array([[1.01501054, 1.        ],
        [0.        , 0.        ]]),
 'V': array([[1.        , 0.87138826],
        [0.87138826, 0.75931751]]),
 'h': 0.0,
 'Pn': array([[-116.65035621,    0.87138826],
        [   0.87138826,    0.75931751]])}

In [543]:
%timeit arima(np.arange(1, 101), order=(1, 0, 1))

4.54 ms ± 70.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
